In [1]:
#INTEL CORPORATION CONFIDENTIAL AND PROPRIETARY
#
#Copyright © 2021 Intel Corporation.
#
#This software and the related documents are Intel copyrighted
#materials, and your use of them is governed by the express
#license under which they were provided to you (License). Unless
#the License provides otherwise, you may not use, modify, copy,
#publish, distribute, disclose or transmit  this software or the
#related documents without Intel's prior written permission.
#
#This software and the related documents are provided as is, with
#no express or implied warranties, other than those that are
#expressly stated in the License.

# Import modules

In [2]:
import os
import sys

import time
import numpy as np
import nxsdk.api.n2a as nx
import nxsdk_modules.slayer.src as nxSlayer
from ntidigits_dataset import NTIDIGITSDataset

os.environ['SLURM'] = '1' 
os.environ['PARTITION'] = 'nahuku32'

# Create nxSlayer Dataset instance

In [3]:
dataset = nxSlayer.auto.s2lDataset(dataset = NTIDIGITSDataset(train = False))

Neuromorphic TIDIGITS dataset is the work of Sensors Group, Institute of 
Neuromoroinformatics, Zurich. It is publicly available under Creative 
Commons Attribution-ShareAlike 4.0 International License here: 
http://sensors.ini.uzh.ch/databases.html

Please cite the following paper if you make use of it.

Jithendar Anumula, Daniel Neil, Tobi Delbruck and Shih-Chii Liu, 
"Feature Representations for Neuromorphic Audio Spike Streams." 
Front. Neurosci., vol. 12, p. 23, 2018.


# Create network

In [4]:
net = nxSlayer.auto.Network('ntidigits.net')

# Compile, setup input-output, and run the network

In [5]:
boardName = 'ntidigits'
loadSavedBoard = False # True # load precompiledBoard or compile again

net.create()

if loadSavedBoard is True:
    net.compile(boardName)
    net.setupIO(dataset, numSnips=3)
else:
    net.compile()
    net.setupIO(dataset, numSnips=3)
    # net.save(boardName) # this will save the compiled board
    
results = net.run()

Creating Network
|   Type   |  W  |  H  |  C  | ker | str | pad |delay|neurons/Core|cores|
|input     |   64|    1|    1|     |     |     |False|        1024|    1|
|dense     |    1|    1|  256|     |     |     |True |         128|    2|
|dense     |    1|    1|  256|     |     |     |True |         128|    2|
|dense     |    1|    1|  242|     |     |     |False|         336|    1|
|average   |    1|    1|   11|     |     |     |False|           0|    0|
|Total                                                            |    6|

Starting compilation ...
Completed compilation in 11.27 seconds
Using per sample spike time: 3000steps (+ 100steps gap)
INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.92:36537
INFO:DRV:      Host server up..............Done 0.43s
INFO:DRV:      Encoding axons/synapses.....Done 0.19s
INFO:DRV:      Compiling Embedded snips....Done 0.65s
INFO:DRV:      Compiling MPDS Registers....Done 0.62ms
INFO:HST:  Config /etc/nx/pio.cfg
INFO

# Gather output population results

In [6]:
results = results.reshape(results.shape[0], 11, -1).sum(axis=2)

# Check accuracy

In [7]:
labels = dataset.labels
accuracy = nxSlayer.s2l.checkAccuracy(labels, results)
print('Final accuracy is {:.2f}%'.format(accuracy*100))

Final accuracy is 92.20%
